In [23]:

import pandas as pd
import numpy as np
import random
from itertools import combinations

# === Kullanıcı Parametreleri ===
CSV_PATH = "value_bets_07092025_gpt.csv"   # Kendi dosya yolunuzu girin
N_COUPONS = 100         # Üretilecek valid kupon sayısı
STAKE_MIN = 5         # Kupon stake eşiği
MAX_ATTEMPTS = 100000   # Valid kupon bulmak için maksimum deneme sayısı
RANDOM_SEED = 42        # Rastgelelik tekrar üretilebilir olsun

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

EXPECTED_COLS = [
    "match_date", "match_time", "match_id", "hometeam", "awayteam",
    "bet_name", "probability", "odds"
]

def _to_float_odds(x):
    """Odds sütununda virgüllü yazımları ("1,15") 1.15'e çevirir."""
    if pd.isna(x):
        return np.nan
    s = str(x).strip().replace('"', '').replace("'", "")
    s = s.replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return np.nan

def kelly_fraction(p: float, odds: float) -> float:
    """Kupon düzeyinde Kelly oranı (f*). odds <= 1 ise 0 döndürür."""
    if pd.isna(p) or pd.isna(odds) or odds <= 1.0 or p <= 0.0 or p >= 1.0:
        return 0.0
    b = odds - 1.0
    q = 1.0 - p
    f_star = (b * p - q) / b
    return max(f_star, 0.0)

def compute_single_bet_metrics(df: pd.DataFrame) -> pd.DataFrame:
    """Referans amaçlı tekil metrikler (kupon hesabında kullanılmaz)."""
    out = df.copy()
    p = out["probability"] / 100.0
    b = out["odds"] - 1.0
    q = 1.0 - p

    out["EV"] = p * out["odds"]
    # Kelly tekil
    out["Kelly"] = np.where(b > 0, ((b * p - q) / b).clip(lower=0.0), 0.0)
    out["Kelly"] = out["Kelly"].fillna(0.0)
    out["stake"] = out["Kelly"] * 100.0

    out["bankroll_if_win"]  = 1.0 + out["Kelly"] * (out["odds"] - 1.0)
    out["bankroll_if_lose"] = 1.0 - out["Kelly"]
    out["expected_bankroll"] = p * out["bankroll_if_win"] + (1.0 - p) * out["bankroll_if_lose"]
    return out


In [24]:

# === 1) Veri Yükleme ve Temizlik ===
raw = pd.read_csv(CSV_PATH)

# Eğer fazladan kolonlar (örn. 'tuttu/yattı') varsa yalnızca beklenen kolonları al.
if set(EXPECTED_COLS).issubset(raw.columns):
    df = raw[EXPECTED_COLS].copy()
else:
    # İlk 8 kolonu al ve başlıkları beklenen isimlerle ayarla (sağlamlık için)
    df = raw.iloc[:, :len(EXPECTED_COLS)].copy()
    df.columns = EXPECTED_COLS

# Tip ve format düzeltmeleri
df["match_id"] = df["match_id"].astype(str)
df["probability"] = pd.to_numeric(df["probability"], errors="coerce")
df["odds"] = df["odds"].apply(_to_float_odds)

# Geçersiz satırları at
df = df.dropna(subset=["match_id", "probability", "odds"])
df = df[df["odds"] > 1.0]
df = df[(df["probability"] >= 0.0) & (df["probability"] <= 100.0)].copy()

print(f"Yüklendi: {len(df)} satır")
df.head(3)


Yüklendi: 11390 satır


,match_date,match_time,match_id,hometeam,awayteam,bet_name,probability,odds
0,2025-09-13,22:30:00,2288891,Famalicao,Sporting CP,Deplasman Her İki Yarıyı da Kazanır mı? :: Hayır,89,1.15
1,2025-09-14,22:00:00,2288066,Juventude,Flamengo,Deplasman Her İki Yarıyı da Kazanır mı? :: Hayır,89,1.14
2,2025-09-14,18:00:00,2284852,AE Kifisia,Panathinaikos,Deplasman Her İki Yarıyı da Kazanır mı? :: Hayır,89,1.20


In [25]:

# === 2) Tekil Bahis Metrikleri (Referans) ===
bets = compute_single_bet_metrics(df)
bets.head(5)


,match_date,match_time,match_id,hometeam,awayteam,bet_name,probability,odds,EV,Kelly,stake,bankroll_if_win,bankroll_if_lose,expected_bankroll
0,2025-09-13,22:30:00,2288891,Famalicao,Sporting CP,Deplasman Her İki Yarıyı da Kazanır mı? :: Hayır,89,1.15,1.0235,0.156667,15.666667,1.0235,0.843333,1.003682
1,2025-09-14,22:00:00,2288066,Juventude,Flamengo,Deplasman Her İki Yarıyı da Kazanır mı? :: Hayır,89,1.14,1.0146,0.104286,10.428571,1.0146,0.895714,1.001523
2,2025-09-14,18:00:00,2284852,AE Kifisia,Panathinaikos,Deplasman Her İki Yarıyı da Kazanır mı? :: Hayır,89,1.20,1.0680,0.340000,34.000000,1.0680,0.660000,1.023120
3,2025-09-13,17:00:00,2288577,Eyüpspor,Galatasaray,Deplasman Her İki Yarıyı da Kazanır mı? :: Hayır,88,1.23,1.0824,0.358261,35.826087,1.0824,0.641739,1.029521
4,2025-09-14,20:15:00,2287905,Haugesund,Rosenborg,Deplasman Her İki Yarıyı da Kazanır mı? :: Hayır,88,1.17,1.0296,0.174118,17.411765,1.0296,0.825882,1.005154


In [ ]:

# === 3) Kupon (3'lü) Üretimi ve Filtre ===

def generate_valid_coupons(bets_df: pd.DataFrame,
                           n_coupons: int = 100,
                           stake_min: float = 5.0,
                           max_attempts: int = 100000,
                           seed: int = 42):
    rng = np.random.default_rng(seed)
    valid = []
    seen_keys = set()
    attempts = 0

    # Index listesinden hızlı örnekleme
    idx = bets_df.index.to_list()

    while len(valid) < n_coupons and attempts < max_attempts:
        attempts += 1

        # 3 satır örnekle (replace=False) — her birinin match_id'si farklı olmalı
        sample_idx = rng.choice(idx, size=3, replace=False)
        sample = bets_df.loc[sample_idx]

        if sample["match_id"].nunique() < 3:
            continue

        # Kupon düzeyi olasılık ve oran
        p_coupon = (sample["probability"] / 100.0).prod()
        odds_coupon = sample["odds"].prod()

        # Kupon EV, Kelly, stake ve bankroll metrikleri (tamamen kupon bazında)
        EV_coupon = p_coupon * odds_coupon
        kelly_coupon = kelly_fraction(p_coupon, odds_coupon)
        stake_coupon = kelly_coupon * 100.0

        bankroll_if_win  = 1.0 + kelly_coupon * (odds_coupon - 1.0)
        bankroll_if_lose = 1.0 - kelly_coupon
        expected_bankroll = p_coupon * bankroll_if_win + (1.0 - p_coupon) * bankroll_if_lose

        # Valid koşulu
        if stake_coupon > stake_min and kelly_coupon > 0:
            # Duplike kuponları engelle (aynı üç maçtan oluşan)
            key = tuple(sorted(sample["match_id"].tolist()))
            if key in seen_keys:
                continue
            seen_keys.add(key)

            valid.append({
                "match_ids": list(key),
                "matches": sample.apply(lambda r: f"{r['hometeam']} - {r['awayteam']}", axis=1).tolist(),
                "bet_names": sample["bet_name"].tolist(),
                "component_probs": (sample["probability"] / 100.0).round(6).tolist(),
                "component_odds": sample["odds"].round(6).tolist(),
                "coupon_probability": round(p_coupon, 8),
                "coupon_odds": round(odds_coupon, 6),
                "coupon_EV": round(EV_coupon, 6),
                "coupon_Kelly": round(kelly_coupon, 6),
                "coupon_stake": round(stake_coupon, 4),
                "coupon_bankroll_if_win": round(bankroll_if_win, 6),
                "coupon_bankroll_if_lose": round(bankroll_if_lose, 6),
                "coupon_expected_bankroll": round(expected_bankroll, 6),
            })

    return valid, attempts

valid_coupons, attempts_used = generate_valid_coupons(
    bets, n_coupons=N_COUPONS, stake_min=STAKE_MIN, max_attempts=MAX_ATTEMPTS, seed=RANDOM_SEED
)
print(f"{len(valid_coupons)} valid kupon bulundu. Deneme sayısı: {attempts_used}")
len(valid_coupons)


In [ ]:
import json

# === 4) Sonuçları Kaydet ===
out_df = pd.DataFrame(valid_coupons)

# Liste kolonlarını JSON stringe çevirerek CSV'ye daha kullanılabilir biçimde yaz
list_cols = ["match_ids", "matches", "bet_names", "component_probs", "component_odds"]
for c in list_cols:
    if c in out_df.columns:
        out_df[c] = out_df[c].apply(lambda x: json.dumps(x, ensure_ascii=False))

OUTPUT_PATH = "valid_coupons.csv"
out_df.to_csv(OUTPUT_PATH, index=False, encoding="utf-8-sig")
print(f"Kaydedildi: {OUTPUT_PATH} — {len(out_df)} satır")
out_df.head(5)


Kaydedildi: valid_coupons.csv — 100 satır


,match_ids,matches,bet_names,component_probs,component_odds,coupon_probability,coupon_odds,coupon_EV,coupon_Kelly,coupon_stake,coupon_bankroll_if_win,coupon_bankroll_if_lose,coupon_expected_bankroll
0,"[""2285184"", ""2285309"", ""2289032""]","[""Vasco Da Gama - Ceara"", ""Arsenal - Not. Fore...","[""Toplam Gol Alt/Üst :: 2,5 Üst"", ""Maç Sonucu ...","[0.55, 0.1, 0.35]","[2.11, 13.15, 3.59]",0.019250,99.609935,1.917491,0.009304,0.9304,1.917491,0.990696,1.008537
1,"[""2282480"", ""2284824"", ""2284832""]","[""Peterhead - Hamilton"", ""Ruthin Town - Llandu...","[""İkinci Yarı Sonucu :: 2.Y 1"", ""İkinci Yarı S...","[0.42, 0.38, 0.18]","[2.73, 2.94, 6.18]",0.028728,49.601916,1.424964,0.008744,0.8744,1.424964,0.991256,1.003716
2,"[""2284891"", ""2285775"", ""2372573""]","[""Huracan - Velez"", ""Cliftonville - Ballymena ...","[""Deplasman Gol Alt/Üst :: Dep 1,5 Üst"", ""İlk ...","[0.4, 0.07, 0.36]","[4.45, 15.3, 2.88]",0.010080,196.084800,1.976535,0.005006,0.5006,1.976535,0.994994,1.004888
3,"[""2286089"", ""2288856"", ""2291332""]","[""Auxerre - Monaco"", ""Juve Stabia - Reggiana"",...","[""Toplam Gol Tek/Çift :: Tek"", ""İlk Yarı Karşı...","[0.6, 0.22, 0.36]","[1.7, 5.24, 3.18]",0.047520,28.327440,1.346120,0.012666,1.2666,1.346120,0.987334,1.004384
4,"[""2285573"", ""2288002"", ""2339656""]","[""Obolon Kiev - Dinamo Kiev"", ""Independiente -...","[""Ev Sahibi Gol Alt/Üst :: Ev 0,5 Üst"", ""İkinc...","[0.56, 0.27, 0.28]","[1.79, 4.74, 3.79]",0.042336,32.156634,1.361383,0.011599,1.1599,1.361383,0.988401,1.004192
